# Part 2: Implementing an MLP classifier

In [0]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 143kB 8.1MB/s 


## Getting Started with a Baseline Multi-Layer Perceptron Model

In addition to being a defacto(Distributed Factorization of Tensors) tensor processing and automatic differentiation library, PyTorch provides a general purpose neural network toolbox. Before we start to look at deep convolutional architectures, we can start with something much simpler - a basic multilayer perceptron. Because the MNIST images are relatively small, a fully connected MLP network will have relatively few weights to train; with bigger images, an MLP might not be practical due to the number of weights.

In this section we will create a simple multi-layer perceptron model with a single hidden layer that achieves an error rate of 1.88%. We will use this as a baseline for comparing more complex convolutional neural network models later.

Let's start off by importing the classes and functions we will need.

In [0]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

When developing, it is always a good idea to initialize the random number generator to a constant to ensure that the results of your script are reproducible each time you run it. Once the model is implemented and tested we might remove this to enable variance to be captured over multiple training runs.

In [0]:
# fix random seed for reproducibility
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import numpy as np
np.random.seed(seed)

## Loading the MNIST data

For the MLP network that we'll shortly define, we need to define a transformation to the data provided by the `MNIST` object that flattens each images into a vector. Before that happens, our transformation must also convert each image from a PIL image to a PyTorch `Tensor`. We can use the classes provided by the `torchvision.transforms` package to compose a set of transforms that given a PIL image will convert the input image to a `Tensor` and reshape that tensor into a vector:

In [0]:
# flatten 28*28 images to a 784 vector for each image
transform = transforms.Compose([
    transforms.ToTensor(),  # convert to tensor
    transforms.Lambda(lambda x: x.view(-1))  # flatten into vector
])

Note that for the latter part of the transform we use `Lambda` which allows us to construct a custom transform (rather than use one provided by torchvision). In this case our lambda function returns a new view of the input tensor which is a vector of length equal to the product of the input tensor's dimensions. In this particular case the input tensor had shape `[28, 28]`, so the output will have shape `[784]`. Passing `-1` as the input to the `view` method makes it automatically compute the output shape on the basis of the input.

Now we can load the training and test splits of the MNIST dataset using the torchvision `MNIST` utility class. We can also provide our `transform` object so that it will be applied automatically when the data loads.

In [0]:
trainset = MNIST(".", train=True, download=True, transform=transform)
testset = MNIST(".", train=False, download=True, transform=transform)

When we train and evaluate our model, we will require batches of data to be provided. As previously mentioned, the `DataLoader` class can automatically provide batches fetched from a `Dataset` object. The `DataLoader` has the ability to fetch batches in the background (using multiple threads if required) to optimise throughput to the deep network model. It also supports shuffling of the data order (very useful for gradient-based learning!).

In [0]:
# create data loaders
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=True)

DataLoaders are iterable and can be used within a loop to fetch batches of data. Each batch is a tuple containing the images in the first element and the labels in the second. __Complete the following code block to calculate the number of instances of each class by iterating over the training data__:

In [0]:
class_counts = torch.zeros(10, dtype=torch.int32)
sizes=[]

for (images, labels) in trainloader:
    # YOUR CODE HERE
    sizes.append(labels.size())
    for l in labels:
      class_counts[l] +=1
    # raise NotImplementedError()

print(max(sizes),min(sizes))
print(class_counts)
print(class_counts.sum())

torch.Size([128]) torch.Size([96])
tensor([5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949],
       dtype=torch.int32)
tensor(60000)


In [0]:
assert class_counts.sum()==60000

__Answer the following questions (enter the answer in the box below each one):__

__1.__ Do all of the batches have the same size?

YOUR ANSWER HERE<br>
No, the max size is 128, the min size is 96


__2.__ How can we configure `DataLoader` to control this?

YOUR ANSWER HERE<br>
By changing the batch_size to b(like 125) which makes mod(60000,b)=0

## Defining the MLP Model

We are now ready to create our simple neural network model. We will define our model in a class that extends `nn.Module`. `nn.Module` subclasses must do a minimum of one thing: implement the `forward` method which takes a batch of data and performs the forward-pass. PyTorch's autograd system will take care of computing the gradients of the forward pass for us. In the code below we'll also make use of the constructor of our model to instantiate the hidden and output layers.

In [0]:
# define baseline model
class BaselineModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(BaselineModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        if not self.training:
            out = F.softmax(out, dim=1)
        return out

The model is a simple neural network with one hidden layer. A rectifier linear unit (ReLU) activation function is used for the neurons in the hidden layer. Note how we use the PyTorch 'Functional API' for stateless operations like applying the ReLU; we could construct a `torch.nn.ReLU` in the constructor and save it in an instance variable instead, but ultimately this would lead to more lines of code.

The `nn.Module` class defines a instance variable called `training` that is set to `True` when the model is being trained and `False` when it is being evaluated after being trained. In our model definition we've used a softmax activation function on the output layer to turn the outputs into probability-like values, but have only set this to be enabled when we are not training the model. We've done this because we will use PyTorch's implementation of Cross Entropy Loss (`nn.CrossEntropyLoss`) during training which implicitly adds a softmax before a logarithmic loss (technically it adds a log-softmax activation followed by a negative log-likelihood loss as this has much better numerical properties). If you've used a library like Keras before you should note that this approach is different to the one taken there - Keras expects you to explicitly add a `Softmax` layer to use with its `categorical_crossentropy` loss.

In our case the softmax isn't actually necessary for model evaluation if we're only interested in the most likely class; the _logits_ (unscaled log probabilities) provided by the final fully connected layer before the softmax can be used directly as the largest logit will correspond to the most likely class.

## Training and Evaluating the Model
We can now fit and evaluate the model. One of the design decisions of PyTorch is that everything should be explicit so we have full control over our models and the training process. This means that we actually need to write the model training loop by hand, and perform each of the various operations (perform the forward-pass, compute the loss, perform the backward-pass, and update the weights). In the code below we'll fit the model to the data over 10 epochs using batches of 128 images provided by the `DataLoader` defined previously. We'll make use of the ADAM optimiser as it broadly tends to work well practically despite its limitations. Running the code below should take just a couple of minutes to complete the training.

In [0]:
# build the model 
model = BaselineModel(784, 784, 10)

# define the loss function and the optimiser
loss_function = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters())

# the epoch loop
for epoch in range(10):
    running_loss = 0.0
    for data in trainloader:
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimiser.zero_grad()

        # forward + loss + backward + optimise (update weights)
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()

        # keep track of the loss this epoch
        running_loss += loss.item()
    print("Epoch %d, loss %4.2f" % (epoch, running_loss))
print('**** Finished Training ****')

Epoch 0, loss 134.76
Epoch 1, loss 51.61
Epoch 2, loss 32.43
Epoch 3, loss 23.45
Epoch 4, loss 17.49
Epoch 5, loss 12.13
Epoch 6, loss 8.55
Epoch 7, loss 7.10
Epoch 8, loss 5.52
Epoch 9, loss 4.28
**** Finished Training ****


In the above we added a crude indicator of progress by printing the total loss at the end of each epoch. Hopefully the loss went down as the model learned! 

At this point it would be good to compute the overall accuracy of the test set. __Use the following code block to finish implementation of the accuracy computation.__ Note that before the code you'll implement we've made a call to `model.eval()` - this sets the model into evaluation mode and supresses non-training things (gradients, and things such as dropout being applied/computed).

In [0]:
model.eval()

# Compute the model accuracy on the test set
correct = 0
total = 0

# YOUR CODE HERE
# raise NotImplementedError()
for data in testloader:
    inputs, labels = data
    outputs = model(inputs)
    for (output, true_label) in zip(outputs, labels):
      total +=1
      if torch.argmax(output)==true_label:
        correct+=1

print('Test Accuracy: %2.2f %%' % ((100.0 * correct) / total))

Test Accuracy: 98.22 %


__In multi-class classification tasks it is often instructive to explore the accuracy of each class. Use the code block below to complete the function that produces the per-class accuracy:__

In [0]:
# Compute the model accuracy on the test set
class_correct = torch.zeros(10)
class_total = torch.zeros(10)

# YOUR CODE HERE
# raise NotImplementedError()
for data in testloader:
    inputs, labels = data
    outputs = model(inputs)
    for (output, true_label) in zip(outputs, labels):
      class_total[true_label] +=1
      if torch.argmax(output)==true_label:
         class_correct[true_label] +=1

for i in range(10):
    print('Class %d accuracy: %2.2f %%' % (i, 100.0*class_correct[i] / class_total[i]))

Class 0 accuracy: 98.78 %
Class 1 accuracy: 99.21 %
Class 2 accuracy: 97.58 %
Class 3 accuracy: 97.72 %
Class 4 accuracy: 98.47 %
Class 5 accuracy: 97.87 %
Class 6 accuracy: 98.23 %
Class 7 accuracy: 98.05 %
Class 8 accuracy: 98.25 %
Class 9 accuracy: 97.92 %
